In [6]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [7]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_c.parquet')

In [8]:
def splitting_def(df):
    date_range_1 = (df.index >= '2020-05-01') & (df.index <= '2020-06-25')
    date_range_2 = (df.index >= '2023-05-01') & (df.index <= '2023-06-15')

    # Combine the date ranges to create the test set
    test_set = df[date_range_1 | date_range_2]

    # The rest of the data will be your training set
    training_set = df[~(date_range_1 | date_range_2)]
    
    # Splitting the test_set into X_test and y_test
    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement']  # Assuming 'pv_measurement' is your target variable

    # Splitting the training_set into X_train and y_train
    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']
    
    return X_train, X_test, y_train, y_test

X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = splitting_def(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

In [9]:
#Create a pool of data

train_pool_c = Pool(X_train_new_c, y_train_new_c)
test_pool_c = Pool(X_test_new_c) 

In [10]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "has_time": trial.suggest_categorical('has-time', [True, False]),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "random_state": 2,
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "nan_mode": trial.suggest_categorical("nan_mode", ["Min", "Max"])
    }

    catboost_model_a = CatBoostRegressor(**params, verbose=100)
    catboost_model_a.fit(train_pool_c)
    pred_c = pd.DataFrame(catboost_model_a.predict(test_pool_c))
    MAE_a = mean_absolute_error(y_test_c, pred_c)
    return MAE_a

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=60)

[I 2023-11-06 20:07:05,625] A new study created in memory with name: no-name-4d8cfa08-f6ca-483c-9622-ee1b532365e1


0:	learn: 152.5279022	total: 52.7ms	remaining: 2m 11s
100:	learn: 110.2013731	total: 3.67s	remaining: 1m 27s
200:	learn: 84.2523050	total: 7.5s	remaining: 1m 25s
300:	learn: 68.9389840	total: 11.4s	remaining: 1m 22s
400:	learn: 60.1151773	total: 15.2s	remaining: 1m 19s
500:	learn: 55.0024534	total: 19.1s	remaining: 1m 16s
600:	learn: 51.9248124	total: 23.1s	remaining: 1m 12s
700:	learn: 49.9826026	total: 27.3s	remaining: 1m 9s
800:	learn: 48.6132168	total: 32.1s	remaining: 1m 7s
900:	learn: 47.5606619	total: 37.2s	remaining: 1m 5s
1000:	learn: 46.6407770	total: 41.8s	remaining: 1m 2s
1100:	learn: 45.8688996	total: 46.3s	remaining: 58.6s
1200:	learn: 45.2235354	total: 51s	remaining: 54.9s
1300:	learn: 44.6538820	total: 55.6s	remaining: 51s
1400:	learn: 44.1632466	total: 1m	remaining: 47.5s
1500:	learn: 43.6900206	total: 1m 6s	remaining: 43.7s
1600:	learn: 43.2423628	total: 1m 11s	remaining: 39.7s
1700:	learn: 42.7700637	total: 1m 16s	remaining: 35.6s
1800:	learn: 42.3225569	total: 1m 21

[I 2023-11-06 20:09:01,857] Trial 0 finished with value: 58.123193703563224 and parameters: {'iterations': 2495, 'learning_rate': 0.004391964575861436, 'depth': 10, 'min_data_in_leaf': 47, 'l2_leaf_reg': 10, 'has-time': True, 'bagging_temperature': 0.5149433339576688, 'random_strength': 0.6706759240891431, 'border_count': 92, 'rsm': 0.8335460689387181, 'nan_mode': 'Min'}. Best is trial 0 with value: 58.123193703563224.


0:	learn: 143.4466740	total: 48.6ms	remaining: 17.3s
100:	learn: 47.8302549	total: 1.49s	remaining: 3.77s
200:	learn: 43.0923448	total: 3.18s	remaining: 2.47s
300:	learn: 39.7828707	total: 4.37s	remaining: 814ms


[I 2023-11-06 20:09:07,127] Trial 1 finished with value: 58.509337406541746 and parameters: {'iterations': 357, 'learning_rate': 0.0795214667949283, 'depth': 6, 'min_data_in_leaf': 84, 'l2_leaf_reg': 5, 'has-time': False, 'bagging_temperature': 0.8468554853484755, 'random_strength': 0.48487033861518813, 'border_count': 78, 'rsm': 0.5106325871115339, 'nan_mode': 'Max'}. Best is trial 0 with value: 58.123193703563224.


...

0:	learn: 151.5756473	total: 32.5ms	remaining: 1m 12s
100:	learn: 70.8252593	total: 7.87s	remaining: 2m 45s
200:	learn: 52.6947313	total: 15.3s	remaining: 2m 33s
300:	learn: 47.9216591	total: 23.8s	remaining: 2m 31s
400:	learn: 45.5090712	total: 31.7s	remaining: 2m 23s
500:	learn: 43.7086681	total: 39.3s	remaining: 2m 15s
600:	learn: 42.1561784	total: 44.5s	remaining: 1m 59s
700:	learn: 40.7452325	total: 49.8s	remaining: 1m 48s
800:	learn: 39.3266490	total: 54.7s	remaining: 1m 37s
900:	learn: 37.9701419	total: 59.7s	remaining: 1m 27s
1000:	learn: 36.7679269	total: 1m 4s	remaining: 1m 19s
1100:	learn: 35.6127709	total: 1m 8s	remaining: 1m 9s
1200:	learn: 34.4920152	total: 1m 11s	remaining: 1m
1300:	learn: 33.4098303	total: 1m 16s	remaining: 54.4s
1400:	learn: 32.3859419	total: 1m 22s	remaining: 48.3s
1500:	learn: 31.4852326	total: 1m 27s	remaining: 42s
1600:	learn: 30.6116983	total: 1m 32s	remaining: 36s
1700:	learn: 29.7930420	total: 1m 38s	remaining: 30.1s
1800:	learn: 29.0623372	tota

[I 2023-11-06 21:28:26,333] Trial 25 finished with value: 57.58854960811515 and parameters: {'iterations': 2222, 'learning_rate': 0.012024209004945042, 'depth': 9, 'min_data_in_leaf': 35, 'l2_leaf_reg': 2, 'has-time': True, 'bagging_temperature': 0.359544791621061, 'random_strength': 0.6768404071773936, 'border_count': 328, 'rsm': 0.23430093784810496, 'nan_mode': 'Max'}. Best is trial 8 with value: 57.03148446122927.


0:	learn: 149.0409508	total: 69.3ms	remaining: 3m 13s
100:	learn: 50.5671171	total: 3.95s	remaining: 1m 45s


[W 2023-11-06 21:28:33,677] Trial 26 failed with parameters: {'iterations': 2793, 'learning_rate': 0.03379294740621279, 'depth': 8, 'min_data_in_leaf': 14, 'l2_leaf_reg': 5, 'has-time': True, 'bagging_temperature': 0.5425054672845713, 'random_strength': 0.7471795673433974, 'border_count': 241, 'rsm': 0.2988416518001317, 'nan_mode': 'Max'} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/tmp/ipykernel_2036/1391305371.py", line 25, in <lambda>
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=60)
  File "/var/tmp/ipykernel_2036/1391305371.py", line 19, in objective
    catboost_model_a.fit(train_pool_c)
  File "/opt/conda/lib/python3.10/site-packages/catboost/core.py", line 5703, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_w

KeyboardInterrupt: 

In [7]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)

with open("optuna-best-parameters_c.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 2004, 'learning_rate': 0.016429110886618033, 'depth': 8, 'colsample_bylevel': 0.7338575760364532, 'min_data_in_leaf': 90, 'l2_leaf_reg': 5, 'has-time': False, 'bagging_temperature': 0.6414768311294344, 'random_strength': 0.4206698081688342, 'border_count': 487, 'rsm': 0.012360675671015169, 'nan_mode': 'min'}
57.13529571086957
